In [16]:
import json
import pandas as pd
from pathlib import Path
from typing import NamedTuple


In [24]:
res_patgh = Path("./Results").resolve()

methods = ["original", "stage_1", "stage_2"]

In [18]:
class Results(NamedTuple):
    reported_cost: float
    lines: pd.DataFrame
    assignments: pd.DataFrame
    flows: pd.DataFrame
    mt_cost: float
    mod_cost: float
    mod_cost_total: float

def get_results(path: Path) -> Results:
    main_result = json.load((path / "metrics.json").open())
    reported_cost = main_result["objective_value"]
    lines = pd.read_csv(path / "used_lines.csv")
    assignments = pd.read_csv(path / "passenger_assignments.csv")
    flows = pd.read_csv(path / "flows.csv")
    mt_cost = lines["line_cost"].sum()
    mod_cost = assignments["mod_cost"].sum()
    mod_cost_total = flows['cost'].sum()
    return Results(
        reported_cost=reported_cost,
        lines=lines,
        assignments=assignments,
        flows=flows,
        mt_cost=mt_cost,
        mod_cost=mod_cost,
        mod_cost_total=mod_cost_total,
    )

# 100% Demand

In [20]:
full_exp_path = res_patgh / "100_percent/budget_10000"
original_path = full_exp_path / 'original'
stage_1_path = full_exp_path / 'stage_1'
stage_2_path = full_exp_path / 'stage_2'


original_results = get_results(original_path)
stage_1_results = get_results(stage_1_path)
stage_2_results = get_results(stage_2_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Workspaces\\Cornell\\line_planning\\Results\\100_percent\\budget_10000\\original\\used_lines.csv'

In [21]:
# crete a dictionary
methods = {
    "Original": original_results,
    "Stage 1": stage_1_results,
    "Stage 2": stage_2_results,
}
methods_selected_results = {}
for method, results in methods.items():
    methods_selected_results[method] = {
        "reported_cost": results.reported_cost,
        "mt_cost": results.mt_cost,
        "mod_cost": results.mod_cost,
        "mod_cost_empty_trips": results.mod_cost_total - results.mod_cost,
        "mod_cost_total": results.mod_cost_total,
    }

# create a dataframe from the dictionary
results_df = pd.DataFrame.from_dict(methods_selected_results, orient='index')
results_df

NameError: name 'original_results' is not defined

# 1% Demand

In [25]:
full_exp_path = res_patgh / "1_percent/budget_10000"
loaded_results = {}
for method in methods:
    loaded_results[method] = get_results(full_exp_path / method)

In [27]:
methods_selected_results = {}
for method, results in loaded_results.items():
    methods_selected_results[method] = {
        "reported_cost": results.reported_cost,
        "mt_cost": results.mt_cost,
        "mod_cost": results.mod_cost,
        "mod_cost_empty_trips": results.mod_cost_total - results.mod_cost,
        "mod_cost_total": results.mod_cost_total,
    }

# create a dataframe from the dictionary
results_df = pd.DataFrame.from_dict(methods_selected_results, orient='index')
results_df

,reported_cost,mt_cost,mod_cost,mod_cost_empty_trips,mod_cost_total
original,11529.0,9847.0,1.000000e+12,1.000000e+12,2.000000e+12
stage_1,67.0,644.0,1.000000e+12,1.000000e+12,2.000000e+12
stage_2,52.0,0.0,1.000000e+12,-1.000000e+12,9.936000e+03
